In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
#nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
import string

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from scipy import stats
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc, roc_curve
from sklearn.metrics import f1_score, classification_report

from sklearn import svm
import gensim.models as g
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import auc, roc_curve, roc_auc_score

import gensim.downloader as api

In [2]:
data = pd.read_csv('/Users/daviddeng8/Documents/Projects/affinity/data/preprocessed_cleaned_body.csv')

In [3]:
right = data.loc[data['Bias'] >= 5]
neutral = data.loc[(data['Bias'] < 5) & (data['Bias'] > -5)]
left = data.loc[data['Bias'] <= -5]
print (len(right) + len(neutral) + len(left))

1675


In [4]:
right['Bias'] = 1
neutral['Bias'] = 0
left['Bias'] = -1

/Users/daviddeng8/anaconda2/envs/practice/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/daviddeng8/anaconda2/envs/practice/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/daviddeng8/anaconda2/envs/practice/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

In [5]:
cat_df = pd.concat([right, neutral, left])
cat_df

,Body,Bias,cleaned_body
13,President Donald Trump has vetoed a resolution...,1,president donald trump ha vetoed resolution st...
155,Asked by if he believes President Trump's emer...,1,asked belief president trump emergency declara...
156,You May Like\nThese 10 Shawshank Redemption Fa...,1,may like shawshank redemption fact insane hive...
157,"NEW: This Germantown, Md. man tells he was att...",1,new germantown md man tell wa attacked saturda...
158,“A little as two to three milligrams of fentan...,1,“a little two three milligram fentanyl induce ...
...,...,...,...
1655,"In any sane reality, last week's vote to hold ...",-1,sane reality last week vote hold new election ...
1656,People have expressed concern that Lindsey Gra...,-1,people expressed concern lindsey graham shamel...
1657,THE DAY OF THE BIGGEST COVER-UP OF ALL IS NIGH...,-1,day biggest coverup nigh thursday anticipating...
1658,"If you are a man or a lady of letters, and are...",-1,man lady letter also versed internet tradition...


In [6]:
info = api.info()
info

{'corpora': {'semeval-2016-2017-task3-subtaskBC': {'num_records': -1,
   'record_format': 'dict',
   'file_size': 6344358,
   'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/semeval-2016-2017-task3-subtaskB-eng/__init__.py',
   'license': 'All files released for the task are free for general research use',
   'fields': {'2016-train': ['...'],
    '2016-dev': ['...'],
    '2017-test': ['...'],
    '2016-test': ['...']},
   'description': 'SemEval 2016 / 2017 Task 3 Subtask B and C datasets contain train+development (317 original questions, 3,169 related questions, and 31,690 comments), and test datasets in English. The description of the tasks and the collected data is given in sections 3 and 4.1 of the task paper http://alt.qcri.org/semeval2016/task3/data/uploads/semeval2016-task3-report.pdf linked in section “Papers” of https://github.com/RaRe-Technologies/gensim-data/issues/18.',
   'checksum': '701ea67acd82e75f95e1d8e62fb0ad29',
   'file_name': 'se

In [7]:
#trained_model = api.load('word2vec-google-news-300')

In [8]:
trained_model = g.KeyedVectors.load_word2vec_format('/Users/daviddeng8/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz', binary = True) 


In [9]:
#trained_model.vector_size
trained_model.vector_size

300

In [10]:
#sample_vec = trained_model['money']
sample_vec = trained_model['money']
sample_vec

array([ 1.58203125e-01,  5.12695312e-02,  6.64062500e-02,  2.10937500e-01,
        3.51562500e-02, -4.66918945e-03, -4.45556641e-03, -8.20312500e-02,
        1.88476562e-01, -1.51367188e-01,  1.58203125e-01,  1.60156250e-01,
       -1.26953125e-01,  1.97265625e-01, -2.61718750e-01,  1.68945312e-01,
       -6.64062500e-02,  1.76757812e-01, -3.54003906e-02,  1.87500000e-01,
        8.44726562e-02,  1.63085938e-01,  1.93359375e-01,  1.30859375e-01,
        6.59179688e-02,  9.52148438e-02, -9.96093750e-02,  1.15234375e-01,
       -1.28906250e-01, -1.22558594e-01, -1.14257812e-01,  2.17285156e-02,
        9.91210938e-02, -7.22656250e-02,  9.57031250e-02, -1.38671875e-01,
        1.52343750e-01, -7.81250000e-03, -5.78613281e-02,  5.24902344e-02,
        1.82617188e-01,  7.66601562e-02,  2.59765625e-01, -3.00781250e-01,
       -3.08593750e-01, -1.68945312e-01, -1.29882812e-01,  2.61718750e-01,
       -2.34375000e-01,  1.40625000e-01,  1.10839844e-01,  7.32421875e-02,
        1.52343750e-01, -

In [11]:
texts = [text for text in cat_df['cleaned_body']]
bodies = [text.split() for text in texts]

In [12]:
###no longer needed
#vector_data = [trained_model[word] for body in bodies for word in body if word in trained_model.vocab]

In [16]:
vector_data = [[trained_model[word] for word in text if word in trained_model.vocab] for text in texts]

In [15]:
vector_data

KeyboardInterrupt: 

In [17]:
len(vector_data)

1675

In [18]:
temp_series = pd.Series(vector_data)

In [19]:
temp_series

0       [[-0.057128906, 0.125, -0.1015625, 0.38085938,...
1       [[-0.29101562, 0.17871094, 0.050048828, 0.0981...
2       [[-0.15332031, 0.16894531, -0.04711914, 0.1328...
3       [[-0.013977051, -0.02722168, -0.05493164, 0.17...
4       [[-0.02722168, 0.23046875, -0.061767578, -0.06...
                              ...                        
1670    [[-0.29101562, 0.17871094, 0.050048828, 0.0981...
1671    [[-0.057128906, 0.125, -0.1015625, 0.38085938,...
1672    [[-0.09082031, 0.25585938, 0.08935547, 0.14843...
1673    [[-0.15332031, 0.16894531, -0.04711914, 0.1328...
1674    [[-0.33789062, 0.19824219, -0.296875, 0.148437...
Length: 1675, dtype: object

In [20]:
cat_df['vectors'] = temp_series

In [21]:
cat_df

,Body,Bias,cleaned_body,vectors
13,President Donald Trump has vetoed a resolution...,1,president donald trump ha vetoed resolution st...,"[[-0.21679688, 0.13574219, 0.18652344, 0.11376..."
155,Asked by if he believes President Trump's emer...,1,asked belief president trump emergency declara...,"[[-0.21679688, 0.13574219, 0.18652344, 0.11376..."
156,You May Like\nThese 10 Shawshank Redemption Fa...,1,may like shawshank redemption fact insane hive...,"[[-0.29101562, 0.17871094, 0.050048828, 0.0981..."
157,"NEW: This Germantown, Md. man tells he was att...",1,new germantown md man tell wa attacked saturda...,"[[-0.06933594, 0.15332031, -0.024902344, 0.172..."
158,“A little as two to three milligrams of fentan...,1,“a little two three milligram fentanyl induce ...,"[[-0.1640625, 0.08203125, 0.028198242, 0.10644..."
...,...,...,...,...
1655,"In any sane reality, last week's vote to hold ...",-1,sane reality last week vote hold new election ...,"[[-0.20800781, 0.034179688, 0.025756836, 0.179..."
1656,People have expressed concern that Lindsey Gra...,-1,people expressed concern lindsey graham shamel...,"[[-0.057128906, 0.125, -0.1015625, 0.38085938,..."
1657,THE DAY OF THE BIGGEST COVER-UP OF ALL IS NIGH...,-1,day biggest coverup nigh thursday anticipating...,"[[-0.23339844, 0.061523438, -0.3046875, 0.2246..."
1658,"If you are a man or a lady of letters, and are...",-1,man lady letter also versed internet tradition...,"[[-0.21679688, 0.13574219, 0.18652344, 0.11376..."


In [23]:
cat_df['vectors']

13      [[-0.21679688, 0.13574219, 0.18652344, 0.11376...
155     [[-0.21679688, 0.13574219, 0.18652344, 0.11376...
156     [[-0.29101562, 0.17871094, 0.050048828, 0.0981...
157     [[-0.06933594, 0.15332031, -0.024902344, 0.172...
158     [[-0.1640625, 0.08203125, 0.028198242, 0.10644...
                              ...                        
1655    [[-0.20800781, 0.034179688, 0.025756836, 0.179...
1656    [[-0.057128906, 0.125, -0.1015625, 0.38085938,...
1657    [[-0.23339844, 0.061523438, -0.3046875, 0.2246...
1658    [[-0.21679688, 0.13574219, 0.18652344, 0.11376...
1659    [[-0.23339844, 0.061523438, -0.3046875, 0.2246...
Name: vectors, Length: 1675, dtype: object

In [24]:
cat_df['vectors'].iloc[0]

[array([-2.16796875e-01,  1.35742188e-01,  1.86523438e-01,  1.13769531e-01,
         1.30004883e-02, -8.59375000e-02, -2.27539062e-01, -6.39648438e-02,
         1.35742188e-01,  1.37695312e-01, -5.24902344e-02, -1.33789062e-01,
        -2.13867188e-01,  8.44726562e-02, -1.02050781e-01, -1.15966797e-02,
        -6.93359375e-02,  1.23046875e-01, -2.00195312e-01, -7.03125000e-02,
        -2.29492188e-01, -5.66406250e-02,  1.61132812e-01,  7.05718994e-04,
        -1.41601562e-01,  1.56250000e-01, -2.27539062e-01,  4.46777344e-02,
         1.80664062e-01,  3.39843750e-01, -8.59375000e-02,  3.66210938e-03,
        -1.66015625e-01,  6.25000000e-02, -2.51953125e-01,  1.35742188e-01,
        -1.45507812e-01,  2.08007812e-01, -9.61914062e-02, -1.15966797e-03,
         2.68554688e-02, -2.25830078e-02,  1.33789062e-01,  4.22363281e-02,
         2.20947266e-02,  7.86132812e-02,  4.41894531e-02, -3.10546875e-01,
        -2.55859375e-01, -6.29882812e-02, -3.59375000e-01,  1.74804688e-01,
        -2.6

In [25]:
#convert into a mean value
mean_vals = [np.mean(text) for text in vector_data]

In [26]:
mean_vals

[-0.0045889188,
 -0.00493934,
 -0.00418685,
 -0.004989782,
 -0.0047147092,
 -0.0047327876,
 -0.0050624455,
 -0.0040428233,
 -0.0046638935,
 -0.004825068,
 -0.0048903855,
 -0.004939298,
 -0.0050230697,
 -0.0053695748,
 -0.0048268912,
 -0.0048217713,
 -0.0050454848,
 -0.004927691,
 -0.005068353,
 -0.0052157147,
 -0.0048594917,
 -0.004847552,
 -0.0047746063,
 -0.005227791,
 -0.004862571,
 -0.0047886427,
 -0.0052665994,
 -0.004834007,
 -0.0050320253,
 -0.004754082,
 -0.005017485,
 -0.00515848,
 -0.004728778,
 -0.0046068286,
 -0.0049184225,
 -0.004867067,
 -0.00496484,
 -0.0047239074,
 -0.0047325403,
 -0.0050923466,
 -0.004876737,
 -0.005047139,
 -0.004645165,
 -0.004775958,
 -0.0047911303,
 -0.0045770262,
 -0.0049144276,
 -0.005103184,
 -0.0043009813,
 -0.004704069,
 -0.0046957005,
 -0.004896238,
 -0.0047884625,
 -0.0053835204,
 -0.0047078314,
 -0.004820424,
 -0.0047097593,
 -0.005038456,
 -0.00500427,
 -0.0051291618,
 -0.005124749,
 -0.0046202764,
 -0.0047712373,
 -0.0049188696,
 -0.00505

In [29]:
#X = cat_df['vectors']
X = pd.Series(mean_vals)
y = cat_df['Bias']

frame = { 'vector': X}
X = pd.DataFrame(frame)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)

In [31]:
X_train

,vector
1583,-0.004702
1485,-0.004884
1449,-0.004744
235,-0.004840
1383,-0.004987
...,...
256,-0.005043
968,-0.005136
952,-0.005206
1667,-0.005492


### unfortunately it looks like we can't take averages since the results are rlly bad

In [32]:
model = svm.SVC(probability = True)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy score: ", accuracy_score(y_test, y_pred))

Accuracy score:  0.4224343675417661


In [33]:
y_probs = model.predict_proba(X_test)
roc_auc = roc_auc_score(y_test, y_probs, average="macro", multi_class="ovo")
print('ROC Score is: ', roc_auc)

ROC Score is:  0.4849029354296548


###  scaling the results

In [34]:
scaler = MinMaxScaler()
scaled = scaler.fit_transform(X)
X = pd.DataFrame(scaled, columns=['average'])
X

,average
0,0.644473
1,0.561992
2,0.739111
3,0.550119
4,0.614865
...,...
1670,0.528393
1671,0.544839
1672,0.461324
1673,0.549264


In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)
X_train

,average
1583,0.617927
1485,0.575119
1449,0.607975
235,0.585490
1383,0.550790
...,...
256,0.537504
968,0.515660
952,0.499248
1667,0.431859


In [36]:
model = svm.SVC(probability = True)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Accuracy score: ", accuracy_score(y_test, y_pred))

Accuracy score:  0.4224343675417661


In [37]:
y_probs = model.predict_proba(X_test)
roc_auc = roc_auc_score(y_test, y_probs, average="macro", multi_class="ovo")
print('ROC Score is: ', roc_auc)

ROC Score is:  0.4807468507571429


### scaling individual vectors with bag of words

In [55]:
import re

def word_extraction(sentence):    
    ignore = ['a', "the", "is"]   
    words = re.sub("[^\w]", " ",  sentence).split()    
    cleaned_text = [w.lower() for w in words if w not in ignore]    
    return cleaned_text

def tokenize(sentences):    
    words = []    
    for sentence in sentences:
        w = word_extraction(sentence)        
        words.extend(w)            
        words = sorted(list(set(words)))    
    return words

def generate_bow(allsentences):        
    vocab = tokenize(allsentences)    
    print("Word List for Document \n{0} \n".format(vocab));

tokenize("hello I'm bob")

['b', 'e', 'h', 'i', 'l', 'm', 'o']

In [68]:
#converts a word directly into its vector format
def vector_form(word):
    if word in trained_model.vocab:
        return trained_model[word]
    else:
        return None

print (vector_form('word'))

[ 3.59375000e-01  4.15039062e-02  9.03320312e-02  5.46875000e-02
 -1.47460938e-01  4.76074219e-02 -8.49609375e-02 -2.04101562e-01
  3.10546875e-01 -1.05590820e-02 -6.15234375e-02 -1.55273438e-01
 -1.52343750e-01  8.54492188e-02 -2.70996094e-02  3.84765625e-01
  4.78515625e-02  2.58789062e-02  4.49218750e-02 -2.79296875e-01
  9.09423828e-03  4.08203125e-01  2.40234375e-01 -3.06640625e-01
 -1.80664062e-01  4.73632812e-02 -2.63671875e-01  9.08203125e-02
  1.37695312e-01 -7.20977783e-04  2.67333984e-02  1.92382812e-01
 -2.29492188e-02  9.70458984e-03 -7.37304688e-02  4.29687500e-01
 -7.93457031e-03  1.06445312e-01  2.80761719e-02 -2.29492188e-01
 -1.91650391e-02 -2.36816406e-02  3.51562500e-02  1.71875000e-01
 -1.12304688e-01  6.25000000e-02 -1.69921875e-01  1.29882812e-01
 -1.54296875e-01  1.58203125e-01 -7.76367188e-02  1.78710938e-01
 -1.72851562e-01  9.96093750e-02  3.94531250e-01  6.44531250e-02
 -6.83593750e-02 -3.18359375e-01  5.95703125e-02 -1.02539062e-02
  9.37500000e-02  8.25195

In [39]:
def text_to_bow_names(text):
    bow = []
    bow = [word for word in text.split() if word not in bow and word in trained_model.vocab]
    return bow
    
text_to_bow_names("david is david is hi piano")



['david', 'is', 'david', 'is', 'hi', 'piano']

In [43]:
frame = {'vectors':temp_series}
vector_df = pd.DataFrame(frame)
vector_df

,vectors
0,"[[-0.057128906, 0.125, -0.1015625, 0.38085938,..."
1,"[[-0.29101562, 0.17871094, 0.050048828, 0.0981..."
2,"[[-0.15332031, 0.16894531, -0.04711914, 0.1328..."
3,"[[-0.013977051, -0.02722168, -0.05493164, 0.17..."
4,"[[-0.02722168, 0.23046875, -0.061767578, -0.06..."
...,...
1670,"[[-0.29101562, 0.17871094, 0.050048828, 0.0981..."
1671,"[[-0.057128906, 0.125, -0.1015625, 0.38085938,..."
1672,"[[-0.09082031, 0.25585938, 0.08935547, 0.14843..."
1673,"[[-0.15332031, 0.16894531, -0.04711914, 0.1328..."


In [50]:
len(vector_df.iloc[0].values)

1

In [47]:
scaler = MinMaxScaler()
scaled = scaler.fit_transform(vector_df)
#vec = pd.DataFrame(scaled, columns=['vectors'])
#X

ValueError: setting an array element with a sequence.